In [ ]:
import sys
sys.path.append("../src")

from extract import load_plan_data, load_postbuy_data

plan_df = load_plan_data()
postbuy_df = load_postbuy_data()

plan_df.head()

In [ ]:
postbuy_df.head()

In [ ]:
postbuy_df.info()

In [ ]:
postbuy_df.isna().sum()

In [ ]:
import pandas as pd

postbuy_clean = postbuy_df[[
    "Date",
    "MarketingInvestment"
]]

postbuy_clean["Date"] = pd.to_datetime(
    postbuy_clean["Date"],
    format="%d/%m/%Y"
)

postbuy_clean = postbuy_clean.dropna(subset=["MarketingInvestment"])

postbuy_daily = (
    postbuy_clean
    .groupby("Date", as_index=False)
    .sum()
)

In [ ]:
postbuy_daily.head()

In [ ]:
postbuy_daily.info()

In [ ]:
# převod Date na datetime (sjednocení s postbuy)
plan_df["Date"] = pd.to_datetime(plan_df["Date"])

# výpočet plánované denní investice dle zadání
plan_df["PlannedInvestment"] = (
    plan_df["Impressions_Client"]
    + plan_df["Clicks_Client"]
    + plan_df["Views_Client"]
)

# vybereme jen relevantní sloupce
plan_daily = plan_df[["Date", "PlannedInvestment"]]

plan_daily.head()

In [ ]:
daily_comparison = plan_daily.merge(
    postbuy_daily,
    on="Date",
    how="left"
)

daily_comparison.head()

In [ ]:
daily_comparison.info()